<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part I, version 0.1,  June 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![titre](images/header.png)

# I - Plan d'échantillonnage SBAE
### Création d'échantillons ponctuels pour analyse de séries
-------

L'estimation de surface basée sur un échantillon part d'un ensemble de points pour une zone d'intérêt (AOI) donnée. Il existe différentes manières de créer un ensemble de points, telles que :

- systématique,
- aléatoire systématique,
- purement aléatoire ou
- aléatoire stratifié.

De plus, la distribution des points est définie par un certain type de géométrie sous-jacente. Il peut s'agir d'une géométrie régulière, appelée **grille** ou d'une **couche de strates** telle qu'une carte d'occupation du sol par exemple. Des grilles régulières peuvent être générées sous les formes suivantes :

- Carrée
- Triangulaire
- Hexagonale

Ce bloc-notes vous guide tout au long du processus de génération d'un tel ensemble de points d'échantillonnage avec les options mentionnées ci-dessus.

### Importer les librairies

*Execute only*

In [1]:
#La première fois, faire tourner cette cellule pour installer les packages manquants
!pip install nrt seaborn retry sepal-ui==2.12.0 dggrid4py

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import time
from datetime import datetime as dt
import matplotlib.pyplot as plt

import ee
import geemap
import geopandas as gpd
from sepal_ui.mapping import SepalMap

# initialize EE    
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

import helpers as h

### 1 - Créer des points d'échantillonnage (semi-) réguliers basés sur des cellules de grille carrées


##### ***Introduction***
Dans cette partie, nous aborderons la création de points d'échantillonnage **systématique** et **systématique aléatoire**.
La routine sous-jacente créera une **grille régulière** d'une taille donnée sur l'AOI donnée.

Sur cette base, il existe 2 options pour créer une couche de points, c'est-à-dire en utilisant le point central de chaque cellule de la grille ou en utilisant un seul emplacement aléatoire dans chaque cellule de la grille.

Lors de la création d'échantillons, une considération importante est la projection utilisée. Les projections présentent toujours des distorsions par rapport à la forme sphérique réelle de la Terre. Comme notre objectif est de donner à chaque échantillon le même poids en termes de représentativité de surface, nous sélectionnerons une projection optimisée pour **surface égale**. Dans le tableau ci-dessous, vous trouverez quelques exemples de projections globales à surface égale parmi lesquelles choisir. Les informations sont tirées d'un article de Yildrim & Kaya 2008 et peuvent être trouvées [ici](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3790990/#:~:text=The%20level%20of%20distortion%20can,compared%20to%20equal%2Darea%20maps). **Notez** que la projection Lat/Lon couramment utilisée (EPSG:4326) **ne** contient pas de distorsions qui doivent être évitées.

| **Projection** | **Code** |
| ------------- |----------------|
| Behrmann | ESRI :54017 |
| Bonne | ESRI :54024 |
| Sinusoïdal | ESRI :54008 |

#### 1.1 - Définir les variables d'entrée

4 variables d'entrée doivent être sélectionnées :

- Projection (comme code epsg/esri)
- AOI (soit en tant que nom de pays, Earth Engine Feature Collection, ou fichier vectoriel lisible par géopandas, c'est-à-dire shapefile, géopackage, geojson, etc.)
- Taille de la grille (longueur de 1 bordure de la cellule de grille carrée)
- Stratégie d'échantillonnage (soit aléatoire pour aléatoire systématique, soit systématique)

In [3]:
# Define the projection to create the grid
projection = 'ESRI:54008'    # Behrmanns equal area

# Define the Area of Interest 
# Example Country case, using FAO's GAUL boundaries
# country = 'Côte d'Ivoire'   
# aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Example using a shapefile (uncomment and adapt path)
aoi_all = gpd.read_file('/home/sepal-user/sbae_point_analysis_CIV/inputs/zae_dissolve_buffer1k.gpkg')
aoi_sel = gpd.read_file('/home/sepal-user/sbae_point_analysis_CIV/inputs/erp_geo.gpkg')

#aoi_sel = aoi_sel.set_crs(4326)

# Define the 
grid_size = 5000 # in metres
sampling_strategy = 'systematic' # either 'random' or 'systematic'

#### 1.2 - Créer la grille et les points d'échantillonnage

In [ ]:
# Créer une grille à l'echelle nationale (14' pour toute Cote d'Ivoire à 1km, 2' pour ERP à 5km)
squared_grid_cells, squared_samples_gdf = h.squared_grid(aoi_sel, grid_size, projection, sampling_strategy)

#squared_grid_cells, squared_samples_gdf = h.squared_grid(aoi_all, grid_size, projection, sampling_strategy)
#squared_samples_gdf = gpd.read_file('/home/dannunzio/sbae_point_analysis/results_square/01_sbae_points.gpkg')

In [ ]:
## Selectionner seulement les points dans la zone ERP
aoi_geom                = aoi_sel.set_crs(4326).to_crs('ESRI:54008')
sel_squared_samples_gdf = gpd.overlay(squared_samples_gdf, aoi_geom)
sel_squared_samples_gdf = sel_squared_samples_gdf.set_crs('ESRI:54008')
sel_squared_samples_gdf

#### 1.3 - Afficher les points

In [ ]:
# Appeler le plot pour l'AOI et les points sélectionner
h.plot_samples(aoi_geom,sel_squared_samples_gdf)

#### 1.4- Enregistrer les sorties

Le notebook vous permet d'exporter la grille finale de 3 manières différentes :
1. Collect Earth Online (format CSV)
2. Fichier vectoriel SIG (format GPKG)
3. Actif Earth Engine (format Feature Collection)

In [ ]:
#------------------------
# define an outputs 
#------------------------

# select on of the above created grids that you want to save
final_samples = sel_squared_samples_gdf

# directory where output files will go
outdir = 'erp_5km' # if left ot None, it goes into module results

# output a CEO compatible csv file
out_ceo_csv = True

# output a Geopackage file
out_gpkg = True

# ee asset
ee_asset_name = 'civ_erp_5km_pts_exo'


### users/fao-dannunzio/civ_1km_pts_sub_erp_exo

# upload to EE
h.upload_to_ee(final_samples, ee_asset_name)

# save to local output directory
h.save_locally(final_samples, ceo_csv=out_ceo_csv, gpkg=out_gpkg, outdir=outdir)

## Nom de l'asset pour la suite
users/fao-dannunzio/civ_erp_5km_pts_exo

### 2 - Créer des points sur une grille hexagonale

***Pourquoi des hexagones ?***

Les hexagones présentent quelques avantages par rapport aux cellules de grille carrées. Avant tout, ils réduisent l'erreur sur la surface, mais ils garantissent également que chaque point dans la cellule de la grille est plus ou moins à la même distance du centre. En effet, la forme idéale selon ce critère serait un cercle. Il n'est cependant pas possible de créer une grille cohérente de cercles. L'hexagone est la forme de polygone qui se rapproche le plus de ce critère, tout en étant capable de fournir un maillage cohérent.

***Système de réseau global discret (DGGS)***

Un système de grille global discret (DGGS) est une série de grilles globales discrètes, généralement constituées de grilles de résolution de plus en plus fines.

***Usage***

Ce bloc-notes fournit une interface simplifiée à la bibliothèque dggrid de [Southern Oregon University](https://www.discreteglobalgrids.org/software/).

Il convient de noter que le DGGRID a un **ensemble fixe de résolutions** qui doit être sélectionné dans la colonne *res* du tableau ci-dessous (vous devez consulter le manuel pour d'autres résolutions si vous sélectionnez une projection différente de l'ISEA3H)

Pour une utilisation avancée, il est recommandé de lire le [manuel](https://webpages.sou.edu/~sahrk/docs/dggridManualV70.pdf).

![title](images/dggrid_isea3h_ap3_res_table.png)


#### 2.1 - Créer une grille hexagonale

In [ ]:
hex_grid_cells, hex_samples_gdf = h.hexagonal_grid(
    aoi=aoi_sel,                      # Area of interest
    resolution=13,                    # refers to the resolution of the grid
    sampling_strategy='systematic',   # choices are 'random' or 'systematic'
    outcrs='EPSG:4326'                # projection of the output data
)

#### 2.2 Plot grid and grid cells

In [ ]:
h.plot_samples(aoi_sel, hex_samples_gdf, hex_grid_cells)

### 3 - Creating a set of irregular sample points based on random and stratified sampling

#### 3.1 - Create points

In [6]:
aoi = aoi_sel
if not isinstance(aoi, ee.FeatureCollection):
    aoi = geemap.geopandas_to_ee(aoi.dissolve())

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = ee.Image(imageCollection.filterBounds(aoi).mosaic().clip(aoi).copyProperties(imageCollection.first()))

stratified_samples = esaLc.stratifiedSample(**{
    "numPoints": 25,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

random_samples = ee.FeatureCollection.randomPoints(**{
    "points": 250,
    "region": aoi,
}).map(h.set_id)

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


random_samples Display in map

**Execute only**

In [8]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi, color=1, width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, {}, 'ESA Worldcover 2020')
Map.addLayer(stratified_samples.draw(color="green", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(random_samples.draw(color="red", pointRadius=2), {}, 'Random Samples per class from ESA LC')

#
Map

EEException: Computation timed out.